In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import os
import random
import numpy as np 
import cv2 
from PIL import Image # Albumentations는 OpenCV 기반이지만, PIL 임포트는 다른 작업에 유용할 수 있음.
import albumentations as A 
from albumentations.pytorch import ToTensorV2 
from tqdm import tqdm # ⭐️ 진행 막대 라이브러리 임포트
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support # ⭐️ 성능 지표 임포트

# ----------------------------------------------------------------------
# 1. ⚙️ 재현성 및 환경 설정
# ----------------------------------------------------------------------

# ⚠️ 실제 환경에 맞게 경로와 크기를 수정하세요.
ROOT_DATA_DIR = '/workspace/merge_data/data/img' 
IMG_SIZE = 64        
BATCH_SIZE = 64      # A40 활용을 위해 배치 크기 증가 (GPU 메모리 허용 내에서 높일수록 유리)
NUM_EPOCHS = 10
SEED = 42
MODEL_NAME = 'cbam_1st.pth'
# GPU 사용 가능 시 "cuda", 아니면 "cpu" 자동 설정
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def set_reproducible_seed(seed_value):
    """모든 난수 생성기의 시드를 고정합니다."""
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed_value)

# ----------------------------------------------------------------------
# 2. 🧠 CBAM 모듈 및 모델 정의
# ----------------------------------------------------------------------

# ChannelAttention, SpatialAttention, CBAM 클래스 코드는 이전과 동일합니다.
class ChannelAttention(nn.Module):
    def __init__(self, in_planes, ratio=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        self.fc = nn.Sequential(nn.Conv2d(in_planes, in_planes // ratio, 1, bias=False),
                                nn.ReLU(),
                                nn.Conv2d(in_planes // ratio, in_planes, 1, bias=False))
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = self.fc(self.avg_pool(x))
        max_out = self.fc(self.max_pool(x))
        out = avg_out + max_out
        return self.sigmoid(out)

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()
        self.conv1 = nn.Conv2d(2, 1, kernel_size, padding=kernel_size//2, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x_out = torch.cat([avg_out, max_out], dim=1)
        x_out = self.conv1(x_out)
        return self.sigmoid(x_out)

class CBAM(nn.Module):
    def __init__(self, in_planes, ratio=16, kernel_size=7):
        super(CBAM, self).__init__()
        self.ca = ChannelAttention(in_planes, ratio)
        self.sa = SpatialAttention(kernel_size)

    def forward(self, x):
        out = x * self.ca(x)
        out = out * self.sa(out)
        return out

class BaselineCBAMModel(nn.Module):
    def __init__(self, num_classes=2):
        super(BaselineCBAMModel, self).__init__()
        self.layer1 = nn.Sequential(nn.Conv2d(3, 16, kernel_size=3, padding=1), nn.BatchNorm2d(16), nn.ReLU(), nn.MaxPool2d(2))
        self.cbam1 = CBAM(16)
        self.layer2 = nn.Sequential(nn.Conv2d(16, 32, kernel_size=3, padding=1), nn.BatchNorm2d(32), nn.ReLU(), nn.MaxPool2d(2))
        self.cbam2 = CBAM(32)
        self.layer3 = nn.Sequential(nn.Conv2d(32, 64, kernel_size=3, padding=1), nn.BatchNorm2d(64), nn.ReLU(), nn.MaxPool2d(2))
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(64, num_classes)

    def forward(self, x):
        x = self.layer1(x)
        x = self.cbam1(x)
        x = self.layer2(x)
        x = self.cbam2(x)
        x = self.layer3(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

# ----------------------------------------------------------------------
# 3. 🖼️ 데이터셋 및 로딩 (Albumentations 적용)
# ----------------------------------------------------------------------

class EmotionDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform 
        self.images = []
        self.label_map = {'happy': 1, 'anger': 0, 'sadness': 0, 'panic': 0}
        
        for emotion_folder in os.listdir(root_dir):
            emotion_path = os.path.join(root_dir, emotion_folder)
            if os.path.isdir(emotion_path) and emotion_folder in self.label_map:
                label = self.label_map[emotion_folder]
                for image_file in os.listdir(emotion_path):
                    if image_file.endswith(('.jpg', '.png', '.jpeg')):
                        self.images.append((os.path.join(emotion_path, image_file), label))
        
        random.shuffle(self.images)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path, label = self.images[idx]
        
        # OpenCV로 이미지 로드 (NumPy 배열, BGR)
        image = cv2.imread(img_path)
        if image is None:
            raise FileNotFoundError(f"Could not read image: {img_path}")
            
        # BGR -> RGB 변환 (Albumentations 표준)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Albumentations 변형 적용
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
            
        return image, label

# ----------------------------------------------------------------------
# 4. 🚀 학습 실행 함수 (GPU 사용 보장 + TQDM 적용)
# ----------------------------------------------------------------------

def train_model():
    # --- 데이터 전처리 (Albumentations Compose 사용) ---
    data_transform = A.Compose([
        A.Resize(IMG_SIZE, IMG_SIZE), 
        # ImageNet 평균/분산으로 정규화
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), 
        ToTensorV2(), 
    ])

    # --- 데이터셋 및 데이터로더 로드 ---
    train_dataset = EmotionDataset(root_dir=os.path.join(ROOT_DATA_DIR, 'train'), transform=data_transform)
    val_dataset = EmotionDataset(root_dir=os.path.join(ROOT_DATA_DIR, 'val'), transform=data_transform)
    
    # 🔥 GPU 전송 가속화 설정
    num_workers = min(os.cpu_count(), 8) # CPU 코어 수 고려
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=num_workers, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=num_workers, pin_memory=True)

    # --- 모델, 손실 함수, 옵티마이저 초기화 ---
    model = BaselineCBAMModel(num_classes=2).to(DEVICE) # 💥 모델을 GPU로 명시적 이동
    criterion = nn.CrossEntropyLoss() 
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    print(f"\n⚡ 학습 장치 설정 완료: {DEVICE}")
    print(f"모델의 첫 파라미터 장치 확인: {next(model.parameters()).device}")

    # --- 학습 루프 ---
    for epoch in range(NUM_EPOCHS):
        # TRAIN
        model.train()
        running_loss = 0.0
        correct_train = 0
        total_train = 0
        
        # ⭐️ tqdm으로 train_loader를 감싸 진행 막대 표시
        train_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{NUM_EPOCHS} [Train]', unit='batch')
        
        for inputs, labels in train_bar:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE) # 💥 데이터를 GPU로 이동
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()
            
            # 실시간 손실 및 정확도 표시 (옵션)
            train_bar.set_postfix({'Loss': loss.item(), 'Acc': f'{100 * correct_train / total_train:.2f}%'})

        epoch_loss_train = running_loss / len(train_dataset)
        epoch_acc_train = correct_train / total_train

        # VALIDATION
        model.eval()
        running_loss_val = 0.0
        correct_val = 0
        total_val = 0
        
        # ⭐️ tqdm으로 val_loader를 감싸 진행 막대 표시
        val_bar = tqdm(val_loader, desc=f'Epoch {epoch+1}/{NUM_EPOCHS} [Val]  ', unit='batch')
        
        with torch.no_grad():
            for inputs, labels in val_bar:
                inputs, labels = inputs.to(DEVICE), labels.to(DEVICE) # 💥 데이터를 GPU로 이동
                
                outputs = model(inputs)
                loss = criterion(outputs, labels)n
                
                running_loss_val += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs.data, 1)
                total_val += labels.size(0)
                correct_val += (predicted == labels).sum().item()
                
                val_bar.set_postfix({'Loss': loss.item()})

        epoch_loss_val = running_loss_val / len(val_dataset)
        epoch_acc_val = correct_val / total_val
        
        # 에폭 최종 결과 출력
        print(f'\n--- Epoch {epoch+1}/{NUM_EPOCHS} Summary ---')
        print(f'  Train Loss: {epoch_loss_train:.4f}, Train Acc: {epoch_acc_train:.4f}')
        print(f'  Val Loss: {epoch_loss_val:.4f}, Val Acc: {epoch_acc_val:.4f}')
        print('-' * 40)
        
        if epoch + 1 == NUM_EPOCHS:
            print(f"\n✅ 최종 에폭 모델 가중치 저장: {MODEL_NAME}")
            torch.save(model.state_dict(), MODEL_NAME)

# ----------------------------------------------------------------------
# 5. 🏃‍♂️ 메인 실행 블록
# ----------------------------------------------------------------------
if __name__ == '__main__':
    # 🚨 Albumentations를 사용하기 위해 'pip install albumentations'와 'pip install tqdm'이 필요합니다.
    set_reproducible_seed(SEED)
    
    if not os.path.exists(os.path.join(ROOT_DATA_DIR, 'train')):
        print("❌ 오류: ROOT_DATA_DIR 설정을 확인하거나, 데이터 폴더(train/val) 구조가 올바른지 확인하세요.")
    else:
        train_model()